In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from gas_storage.gas_price_simulations import GasPriceSimulations

##### Ingest prices

In [ ]:
simulator = GasPriceSimulations()
paths = simulator.get_simulations()
fig = go.Figure()
for path in paths[:50, :]:
    fig.add_trace(
        go.Scatter(
            x=simulator.index, y=path, showlegend=False, line=dict(color="lightgray")
        )
    )
fig = fig.update_layout(template="plotly_dark", title="Gas spot price paths")
# fig.show()

### Storage definition
$$
\begin{align*}
I_t \in [0, I_{max}] &\dots \text{inventory} \\
q_{in},\;q_{out} &\dots \text{max injection and withdrawal rate} \\
S_t &\dots \text{gas spot price} \\
a_t \in [-q_{out},q_{in}] &\dots \text{inventory action}\\
-a_t\cdot S_t &\dots CF_t,\;\text{cash flow} \\
\end{align*}
$$

##### Inventory dynamics
$$
I_{t+1} = I_t + a_t \\
$$

##### Terminal condition
$$
V_T(I) =I_T\cdot S_T,\quad T\dots \text{final time}
$$

##### Objective function
$$
\max_{a_0,\dots,a_{T-1}}\mathbb{E}[\sum^{T-1}_{t=0}(-a_t\cdot S_t)+I_T\cdot S_T]
$$

### Bellman recursion
$$
\begin{align*}
V_t(I) &= \max_{a\in \mathcal{A}(I_t)}(-a\cdot S_t + \mathbb{E}[V_{t+1}(I+a)\mid S_t]) \\
\end{align*}
$$
Value today equals - best chosen action - immediate cashflow - plus expected value of tomorrow.

##### Expectation handling
$$
\mathbb{E}[V_{t+1}(T_{t+1})\mid S_t,I_t]
$$
What is the average future value of the storage tomorrow, given what I know today: today's price $S_t$ and today's inventory $I_t$.

### Example
$$
\begin{align*}
t &\dots \text{today} \\
S_t &= 30\; \text{EUR/MWh} \\
I_t &= 50\; \text{MWh} \\
a_t &\in \{-5, 0, 5\} \\
V_t(I_t) &= \text{Maximum expected future profit from today onward} \\
\end{align*}
$$

##### Action 1 - inject
$$
\begin{align*}
I_{t+1} &= 50+5=55 \\
CF_t &= -5\cdot 30 = -150 \\
V_{t+1}(55) &= \text{?} \\
\text{value} &: −150+\mathbb{E}[V_{t+1}​(55) \mid S_t​=30,I_t​=50]
\end{align*}
$$

##### Action 2 - hold
$$
\begin{align*}
I_{t+1} &= 50 \\
CF_t &= 0 \\
\text{value} &: \mathbb{E}[V_{t+1}​(50) \mid S_t​=30,I_t​=50]
\end{align*}
$$

##### Action 3 - withdraw
$$
\begin{align*}
I_{t+1} &= 45 \\
CF_t &= -(-5)\cdot 30 = 150 \\
\text{value} &: 150+\mathbb{E}[V_{t+1}​(45) \mid S_t​=30,I_t​=50]
\end{align*}
$$

##### Decision
$$
\begin{align*}
\text{inject} &: −150+\mathbb{E}[V_{t+1}​(55) \mid S_t​=30,I_t​=50] \\
\text{hold} &: \mathbb{E}[V_{t+1}​(50) \mid S_t​=30,I_t​=50] \\
\text{withdraw} &: 150+\mathbb{E}[V_{t+1}​(45) \mid S_t​=30,I_t​=50] \\
\end{align*}
$$

In [ ]:
I_max = 100.0
q_in = 5.0
q_out = 5.0

n_I = 51
I_grid = np.linspace(0.0, I_max, n_I)


n_paths, n_steps = paths.shape

V = np.zeros((n_paths, n_I))
S_T = paths[:, -1]

for i, I in enumerate(I_grid):
    V[:, i] = I * S_T

actions = np.array([-q_out, 0.0, q_in])

In [ ]:
def continuation_value(S, Y):
    # regress Y on [1, S, S^2]
    X = np.column_stack([np.ones_like(S), S, S**2])
    beta, _, _, _ = np.linalg.lstsq(X, Y, rcond=None)
    return X @ beta

In [ ]:
for t in reversed(range(n_steps - 1)):
    print(t)
    S_t = paths[:, t]
    V_next = V.copy()

    for i, I in enumerate(I_grid):
        values = []

        for a in actions:
            I_new = I + a
            if I_new < 0 or I_new > I_max:
                values.append(np.full_like(S_t, -np.inf))
                continue

            j = np.argmin(np.abs(I_grid - I_new))

            cont = continuation_value(S_t, V_next[:, j])
            payoff = -a * S_t + cont
            values.append(payoff)

        V[:, i] = np.max(np.column_stack(values), axis=1)

In [ ]:
V.shape

In [ ]:
I0 = 50.0
i0 = np.argmin(np.abs(I_grid - I0))

storage_value = V[:, i0].mean()
print(f"Estimated storage value: {storage_value:.2f}")